In [1]:
%%writefile going_modular/get_data.py

import os
import requests
import zipfile 
from pathlib import Path

# setup the data path
data_path = Path("data/")
image_path = data_path / 'pizza_steak_sushi'

if image_path.is_dir():
	print(f"{image_path} already exists.")
else:
	print(f"Did not found any {image_path} directory, creating one...")
	image_path.mkdir(parents=True, exist_ok=True)

# Downloading pizza, sushi, steak
with open(data_path / "pizza_steak_sushi.zip", "wb") as file:
	response = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
	print(f"Downloading pizza, steak, sushi...")
	file.write(response.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
	print(f"Unzipping pizza, steak, sushi...")
	zip_ref.extractall(image_path)

# remove the zip file
os.remove(data_path / 'pizza_steak_sushi.zip')

Writing going_modular/get_data.py


In [3]:
train_dir = image_path / 'train'
test_dir = image_path / 'test'

In [33]:
from torchvision import transforms

transform = transforms.Compose([
	transforms.Resize((64, 64)),
	transforms.ToTensor()
])

In [34]:
import torch
from going_modular import data_setup, model_builder, engine

device = "cuda" if torch.cuda.is_available() else "cpu"

# Create train/test dataloaders and get the class_names list
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
	train_dir=train_dir,
	test_dir=test_dir,
	transform=transform,
	batch_size=32
)

torch.manual_seed(42)
model_v01 = model_builder.TinyVGG(
	input_shape=3,
	hidden_units=10,
	output_shape=len(class_names)
).to(device)



In [35]:
NUM_EPOCHS = 4

In [36]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

In [37]:
engine.train(
	model=model_v01,
	train_dataloader=train_dataloader,
	test_dataloader=test_dataloader,
	optimizer=optimizer,
	loss_fn=loss_fn,
	epochs=NUM_EPOCHS,
	device=device 
)

 25%|██▌       | 1/4 [00:15<00:45, 15.13s/it]

Epoch: 1 | train_loss: 1.0956 | train_acc: 0.4023 | test_loss: 1.0898 | test_acc: 0.5417


 50%|█████     | 2/4 [00:28<00:27, 13.97s/it]

Epoch: 2 | train_loss: 1.1035 | train_acc: 0.2812 | test_loss: 1.0898 | test_acc: 0.5417


 75%|███████▌  | 3/4 [00:41<00:13, 13.81s/it]

Epoch: 3 | train_loss: 1.0996 | train_acc: 0.2812 | test_loss: 1.0898 | test_acc: 0.5417


100%|██████████| 4/4 [00:56<00:00, 14.13s/it]

Epoch: 4 | train_loss: 1.0997 | train_acc: 0.2812 | test_loss: 1.0898 | test_acc: 0.5417


{'train_loss': [1.095619022846222,
  1.1035260558128357,
  1.0996392220258713,
  1.0997427254915237],
 'train_acc': [0.40234375, 0.28125, 0.28125, 0.28125],
 'test_loss': [1.089814305305481,
  1.089814305305481,
  1.089814305305481,
  1.089814305305481],
 'test_acc': [0.5416666666666666,
  0.5416666666666666,
  0.5416666666666666,
  0.5416666666666666]}

In [40]:
from going_modular import utils

# Save a model to a file
utils.save_model(
	model=model_v01,
	target_dir="./model",
	model_name="Model_Sushi_Pizza_Steak.pth"
)

[INFO] Saving model to: model\Model_Sushi_Pizza_Steak.pth


In [41]:
%%writefile going_modular/train.py

"""
	Trains a PyTorch image classification model using device-agnosic code
"""

import os
import torch
import data_setup, engine, model_builder, utils

from torchvision import transforms

# Hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

data_transform = transforms.Compose(
	transforms.Resize((64, 64)),
	transforms.ToTensor()
)

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
	train_dir=train_dir,
	test_dir=test_dir,
	transform=data_transform,
	batch_size=BATCH_SIZE
)

# model
model = model_builder.TinyVGG(
	input_shape=3,
	hidden_units=HIDDEN_UNITS,
	output_shape=len(class_names)
).to(device)

# loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(
	model=model_v01,
	train_dataloader=train_dataloader,
	test_dataloader=test_dataloader,
	optimizer=optimizer,
	loss_fn=loss_fn,
	epochs=NUM_EPOCHS,
	device=device 
)

# Save a model with help from utils.py
utils.save_model(
	model=model_v01,
	target_dir="./model",
	model_name="Model_Sushi_Pizza_Steak.pth"
)

Writing going_modular/train.py


In [4]:
import argparse

parser = argparse.ArgumentParser(
                    prog='ProgramName',
                    description='What the program does',
                    epilog='Text at the bottom of help')

print(parser.add_argument('-c', '--count'))      # option that takes a value


_StoreAction(option_strings=['-c', '--count'], dest='count', nargs=None, const=None, default=None, type=None, choices=None, required=False, help=None, metavar=None)


In [40]:
%%writefile prediction.py

import torch
import torchvision
from torchvision import transforms
from typing import List
import matplotlib.pyplot as plt
import model_builder

data_transform = transforms.Compose(
	[transforms.Resize((64, 64))]
)

device = "cuda" if torch.cuda.is_available() else "cpu"
class_names = ['pizza', 'steak', 'sushi']
MODEL_DICT_PATH = "model/Model_Sushi_Pizza_Steak.pth"
EXAMPLE_PATH = "./example.jpg"
model = model_builder.TinyVGG(input_shape=3, hidden_units=10, output_shape=3).to(device)
model_state_dict = torch.load(MODEL_DICT_PATH, weights_only=True)
model.load_state_dict(model_state_dict)

def pred_and_plot_image(model: torch.nn.Module,
						image_path: str,
						class_names: List[str],
						transform=None,
						device: torch.device = device,
						):

	target_image = torchvision.io.read_image(str(image_path)).type(torch.float32)

	if transform:
		target_image = transform(target_image)

	target_image = target_image / 255

	model.to(device)

	model.eval()
	with torch.inference_mode():
		target_image = target_image.unsqueeze(dim=0)

		target_image_pred = model(target_image.to(device))
	
	target_image_pred_probs = torch.softmax(target_image_pred, dim=1)
	target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)

	plt.imshow(target_image.squeeze().permute(1, 2, 0)) # make sure it's the right size for matplotlib
	if class_names:
	    title = f"Prediction: {class_names[target_image_pred_label.cpu()]} | Probability: {target_image_pred_probs.max().cpu():.3f}/1"
	else: 
		title = f"Prediction: {target_image_pred_label} | Probability: {target_image_pred_probs.max().cpu():.3f}/1"
	plt.title(title)
	plt.axis(False)
	plt.show()

pred_and_plot_image(
	model=model,
	image_path=EXAMPLE_PATH,
	class_names=class_names,
	device=device,
	transform=data_transform,
)

Writing prediction.py


In [ ]:
# `predict.py`

base_path = "./example.jpg"

